In [33]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder \
    .appName("Delta Spark 4.0.0") \
    .master("local[*]") \
    .enableHiveSupport() \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [34]:
#CARGANDO INFORMACIÓN EN DATAFRAMES
dim_ventas = spark.read.format("delta").load("../sql/dw_silver/dim_ventas")
dim_productos = spark.read.format("delta").load("../sql/dw_silver/dim_productos")
dim_clientes = spark.read.format("delta").load("../sql/dw_silver/dim_clientes")
dim_territorio = spark.read.format("delta").load("../sql/dw_silver/dim_territorio")
dim_fecha = spark.read.format("delta").load("../sql/dw_silver/dim_fecha")

In [35]:
dim_ventas.show(5)
dim_productos.show(5)
dim_clientes.show(5)
dim_territorio.show(5)
dim_fecha.show(5)

+--------+---------+------------+-------------------+----------+---------------+---------------+-----------+
|PedidoID|ClienteID|TerritorioID|        FechaPedido|ProductoID|Cantidad_pedido|Precio_Unitario|Linea_total|
+--------+---------+------------+-------------------+----------+---------------+---------------+-----------+
|   61793|    21540|          10|2013-12-09 00:00:00|       980|              1|         769.49|     769.49|
|   62015|    20437|           4|2013-12-12 00:00:00|       799|              1|        1120.49|    1120.49|
|   62680|    29168|           4|2013-12-23 00:00:00|       978|              1|         742.35|     742.35|
|   62985|    12785|           6|2013-12-28 00:00:00|       782|              1|        2294.99|    2294.99|
|   63087|    13861|           1|2013-12-30 00:00:00|       878|              1|          21.98|      21.98|
+--------+---------+------------+-------------------+----------+---------------+---------------+-----------+
only showing top 5 

In [36]:
#CREACIÓN DE TABLA DE HECHOS
from pyspark.sql.functions import date_format, col
df_hechos = dim_ventas.select(
    "ProductoID",
    "PedidoID",
    date_format(col("FechaPedido"), "yyyyMMdd").cast("int").alias("FechaID"),
    "ClienteID",
    "TerritorioID",
    "Cantidad_pedido",
    "Linea_total"
)
df_hechos.show()

+----------+--------+--------+---------+------------+---------------+-----------+
|ProductoID|PedidoID| FechaID|ClienteID|TerritorioID|Cantidad_pedido|Linea_total|
+----------+--------+--------+---------+------------+---------------+-----------+
|       980|   61793|20131209|    21540|          10|              1|     769.49|
|       799|   62015|20131212|    20437|           4|              1|    1120.49|
|       978|   62680|20131223|    29168|           4|              1|     742.35|
|       782|   62985|20131228|    12785|           6|              1|    2294.99|
|       878|   63087|20131230|    13861|           1|              1|      21.98|
|       797|   63106|20131230|    25029|           9|              1|    1120.49|
|       975|   63155|20131231|    30094|           4|              2|   2041.188|
|       875|   63271|20131231|    30022|           4|              6|     32.364|
|       871|   63574|20140104|    20533|           4|              1|       9.99|
|       871|   6

In [37]:
df_hechos.write.option("mergeSchema", "true").format("delta").mode("overwrite").save("../sql/dw_gold/df_hechos")

In [38]:
#OPERACIONES DE CONSULTAS
from pyspark.sql.functions import sum as _sum, round

#SUMA DE VENTAS POR CONTINENTE
df_ventas_continente = df_hechos.join(
    dim_territorio,
    on = "TerritorioID",
    how = "inner"
).groupBy("Continente").agg(
    round(_sum("Linea_total"), 2).alias("Suma_Total")
)
df_ventas_continente.show()

#CANTIDAD VENDIDA POR PRODUCTO - AÑADIR TIPO O CLASIFICACIÓN DE PRODUCTO
df_cant_vendida = df_hechos.join(
    dim_productos,
    on = "ProductoID",
    how = "Inner"
).groupBy("Nombre_producto").agg(
    round(_sum("Precio_lista"), 2).alias("Suma_Total")
)
df_cant_vendida.show()

#TOP 10 PRODUCTOS MÁS VENDIDOS - AÑADIR TIPO O CLASIFICACIÓN DE PRODUCTO
df_top = df_hechos.join(
    dim_productos,
    on = "ProductoID",
    how = "inner"
).groupBy("Nombre_producto").agg(
    round(_sum("Cantidad_pedido"),2).alias("Suma_Cantidad")
).orderBy("Suma_Cantidad", ascending = False).limit(10)
df_top.show()

#VENTAS AGRUPADAS POR CONTINENTE
df_ventas_agrupadas = df_hechos.join(
    dim_territorio,
    on = "TerritorioID",
    how = "inner"
).join(
    dim_productos,
    on = "ProductoID",
    how = "Inner"
).groupBy("Continente","Nombre_producto").agg(round(_sum("Linea_total"),2).alias("Costo_Ventas_Continente"))
df_ventas_agrupadas.show()

#VENTAS REALIZADAS POR CONTINENTES POR FECHA
df_ventas_fecha = df_hechos.join(
    dim_fecha,
    on = "FechaID",
    how = "Inner"
).join(
    dim_territorio,
    on = "TerritorioID",
    how = "inner"
).groupBy("Continente","Fecha").agg(round(_sum("Linea_total"),2).alias("Ventas"))
df_ventas_fecha.show()

+-------------+-------------+
|   Continente|   Suma_Total|
+-------------+-------------+
|       Europe|   9180532.57|
|North America|1.482072734E7|
|      Pacific|   8979109.04|
+-------------+-------------+

+--------------------+----------+
|     Nombre_producto|Suma_Total|
+--------------------+----------+
| HL Road Frame - Red|  32139.47|
|  ML Road Rear Wheel|    826.16|
|  LL Road Handlebars|     445.4|
|  Mountain-100 Black| 597899.74|
|         ML Crankset|    256.49|
|       Touring Pedal|    323.96|
|      Women's Tights|    1577.6|
|      Road-250 Black|1914140.86|
|          LL Headset|     205.2|
|LL Mountain Frame...|   4482.41|
|   Road-550-W Yellow|1061775.82|
|          HL Headset|    997.84|
|HL Touring Handle...|    549.42|
|  Full-Finger Gloves|   1551.42|
|Mountain Bottle Cage|   6483.51|
|ML Mountain Handl...|   2105.28|
|ML Mountain Seat/...|    273.98|
|   Touring-3000 Blue| 153199.67|
|HL Touring Frame ...|  15645.34|
|LL Touring Seat/S...|     27.12|
+------